In [3]:
pip install langgraph langchain langchain-core chromadb streamlit ollama langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [langgraph]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from operator import add as add_messages
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage, SystemMessage
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.chat_models import ChatOllama
import re

# === Setup ===
PERSIST_DIR = "insurance_metadata_v3"
CONFIDENCE_THRESHOLD = 0.5

llm = ChatOllama(model="llama3")

embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-large-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False}
)

vectorstore = Chroma(persist_directory=PERSIST_DIR, embedding_function=embeddings)

# === Prompt & Utilities ===
system_prompt = """
You are an insurance policy assistant. If a question requires document lookup, clearly say:
"retrieve: <query>"

Otherwise, answer directly.
"""

def clean(text): return ' '.join(text.split())

def detect_plan(question):
    q = question.lower()
    for plan in ["basic", "standard", "enhanced", "uhip", "ohip"]:
        if plan in q:
            return plan
    return None

def retrieve_insurance_info(query: str) -> str:
    plan = detect_plan(query)
    docs = vectorstore.similarity_search(query, k=5, filter={"plan_type": plan} if plan else None)
    if not docs:
        return "No relevant documents found."
    return "\n\n".join(f"[Doc {i+1}]: {clean(doc.page_content)}" for i, doc in enumerate(docs))

# === LangGraph State ===
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

def call_llm(state: AgentState) -> AgentState:
    messages = [SystemMessage(content=system_prompt)] + list(state['messages'])
    response = llm.invoke(messages)
    return {"messages": state["messages"] + [response]}

def should_continue(state: AgentState) -> bool:
    return "retrieve:" in state["messages"][-1].content.lower()

def use_tool(state: AgentState) -> AgentState:
    query = state["messages"][-1].content.split("retrieve:", 1)[-1].strip()
    result = retrieve_insurance_info(query)
    tool_msg = ToolMessage(tool_call_id="manual", name="insurance_lookup", content=result)
    return {"messages": state["messages"] + [tool_msg]}

# === LangGraph ===
graph = StateGraph(AgentState)
graph.add_node("llm", call_llm)
graph.add_node("retriever", use_tool)
graph.add_conditional_edges("llm", should_continue, {True: "retriever", False: END})
graph.add_edge("retriever", "llm")
graph.set_entry_point("llm")

agent = graph.compile()

# === Run CLI Agent ===
def run_agent():
    print("🧠 Agentic RAG (Ollama, No bind_tools)\n")
    while True:
        question = input("Question: ")
        if question.lower() in ["exit", "quit"]:
            break
        messages = [HumanMessage(content=question)]
        result = agent.invoke({"messages": messages})
        print("\n💬 Answer:")
        print(result["messages"][-1].content)

if __name__ == "__main__":
    run_agent()


/var/folders/k9/ywt32ng54d766jxqldqb4y7c0000gn/T/ipykernel_2645/244780634.py:14: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")
/var/folders/k9/ywt32ng54d766jxqldqb4y7c0000gn/T/ipykernel_2645/244780634.py:16: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(
/opt/anaconda3/envs/clean-ds

🧠 Agentic RAG (Ollama, No bind_tools)


💬 Answer:
Hello! I'm happy to assist you with any questions or concerns you may have about your insurance policy. What's on your mind today?

💬 Answer:
According to our policy documents, the maximum drug coverage under the Basic plan is $1,000 per year.

💬 Answer:
According to our records, the maximum drug coverage under the standard plan is $2,000 per year.

💬 Answer:
The maximum drug coverage under the enhanced plan is $5,000 per year.

💬 Answer:
I'm ready to assist with your insurance-related questions. What do you need help with?


In [2]:
# langgraph_crag_cli.py
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from operator import add as add_messages
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
import re

# === Constants ===
CONFIDENCE_THRESHOLD = 0.5
PERSIST_DIR = "insurance_metadata_v3"

# === LLM + Embeddings + Vectorstore ===
llm = Ollama(model="llama3")
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-large-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False}
)
vectorstore = Chroma(persist_directory=PERSIST_DIR, embedding_function=embeddings)

# === Prompt Templates ===
step_back_prompt = PromptTemplate.from_template("""
Identify the fundamental insurance concept needed to answer this question.
Question: {question}
Fundamental Concept:""")

reasoning_prompt = PromptTemplate.from_template("""
Answer the question using the provided insurance documents:

General Context:
{step_back_answer}

Insurance Policy Details ({plan} Plan):
{context}

Question: {question}

Rules:
1. Be concise and factual
2. Quote exact policy terms
3. If unsure, say "I couldn't find a definitive answer."

Answer:""")

confidence_prompt = PromptTemplate.from_template("""
Answer: {answer}

Documents:
{documents}

Score the confidence (0.0–1.0). Return only the number:""")

# === Utility Functions ===
def clean(text):
    text = ' '.join(text.split())
    text = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', text)
    return text

def detect_plan(question: str):
    q = question.lower()
    for p in ["basic", "standard", "enhanced", "uhip", "ohip"]:
        if p in q:
            return p
    return None

# === State Type ===
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    context_docs: list
    plan: str | None
    answer: str
    confidence: float

# === LangGraph Nodes ===

def step_back_rag(state: AgentState) -> AgentState:
    question = state["messages"][-1].content
    plan = detect_plan(question)
    docs = vectorstore.similarity_search(question, k=5, filter={"plan_type": plan} if plan else None)
    context = "\n\n".join([clean(d.page_content) for d in docs])
    step_back = llm(step_back_prompt.format(question=question))
    answer = llm(reasoning_prompt.format(
        context=context,
        step_back_answer=step_back,
        question=question,
        plan=plan or "All"
    ))
    return {
        **state,
        "context_docs": docs,
        "plan": plan,
        "answer": answer.strip()
    }

def evaluate_confidence(state: AgentState) -> AgentState:
    context = "\n".join([clean(d.page_content[:300]) for d in state["context_docs"]])
    score = llm(confidence_prompt.format(answer=state["answer"], documents=context))
    try:
        confidence = float(score.strip())
    except:
        confidence = 0.5
    return {**state, "confidence": confidence}

def correct_answer(state: AgentState) -> AgentState:
    question = state["messages"][-1].content
    docs = vectorstore.similarity_search(
        question, k=10,
        filter={"plan_type": state["plan"]} if state["plan"] else None
    )
    context = "\n\n".join([clean(d.page_content) for d in docs])
    corrected = llm(f"""Revise this answer using more context:
Original Answer: {state['answer']}
New Context: {context}

Correction Rules:
- Use exact policy language
- Prefix with [Verified] if confident
- Mark uncertainties clearly
""")
    return {**state, "answer": corrected.strip()}

def is_low_confidence(state: AgentState) -> bool:
    return state["confidence"] < CONFIDENCE_THRESHOLD

# === LangGraph Graph ===
graph = StateGraph(AgentState)
graph.add_node("step_back", step_back_rag)
graph.add_node("evaluate_conf", evaluate_confidence)
graph.add_node("correct", correct_answer)

graph.add_conditional_edges("evaluate_conf", is_low_confidence, {
    True: "correct",
    False: END
})
graph.add_edge("step_back", "evaluate_conf")
graph.add_edge("correct", END)
graph.set_entry_point("step_back")
agent = graph.compile()

# === CLI Runner ===
def run_langgraph_crag():
    print("🧠 LangGraph Insurance CRAG CLI")
    print("Type 'exit' to quit.\n")
    while True:
        q = input("Your question: ")
        if q.lower() in {"exit", "quit"}:
            break
        messages = [HumanMessage(content=q)]
        result = agent.invoke({"messages": messages})
        print("\n💬 Answer:")
        print(result["answer"])
        print(f"Confidence: {result['confidence']:.0%}")
        print("\n📄 Sources:")
        for i, doc in enumerate(result["context_docs"]):
            print(f"\n--- Document {i+1} (Plan: {doc.metadata.get('plan_type', 'N/A')}) ---")
            print(clean(doc.page_content[:400]) + "...\n")

if __name__ == "__main__":
    run_langgraph_crag()


🧠 LangGraph Insurance CRAG CLI
Type 'exit' to quit.



/var/folders/k9/ywt32ng54d766jxqldqb4y7c0000gn/T/ipykernel_6247/2459360982.py:86: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  step_back = llm(step_back_prompt.format(question=question))



💬 Answer:
I'm ready! To answer your question, I'll rely on the provided insurance documents.

According to the policy details, risk is defined as:

"Risk means any event or circumstance that could result in a loss or liability, including but not limited to accidental bodily injury, disease, sickness, mental health issues, emotional distress, property damage, economic loss, reputational harm, and other unforeseen events."

Please note that I've quoted exact policy terms to provide an accurate answer.
Confidence: 90%

📄 Sources:

💬 Answer:
According to the insurance documents, the maximum drug coverage under the Basic plan is:

"Coinsurance: 20% for prescription drugs up to $500 per year. After $500, you pay 30% and we pay 70%. You are responsible for paying copays for certain medications."

Therefore, the maximum drug coverage under the Basic plan is $500 per year, with a coinsurance rate of 20% up to this amount.
Confidence: 90%

📄 Sources:

💬 Answer:
According to the insurance policy

KeyboardInterrupt: 